In [1]:
# import re
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [4]:
!ls -1 data

avito_2022-09-01_14-07_location.pkl
avito_2022-09-01_14-07_raw.xlsx
avito_2022-09-01_14-07.xlsx
old


In [5]:
ts = '2022-09-01_14-07'

In [6]:
df = pd.read_excel(f'data/avito_{ts}.xlsx').merge( pd.read_pickle(f'data/avito_{ts}_location.pkl'), on='adr')
print(len(df))
df.sample(3)

3953


,adr,obj_name,title,priceM,nrooms,floor,nfloors,area,is_studio,is_apartment,...,is_openspace,is_SNT,is_last_floor,is_roof,description,price,avito_page,avito_id,coo,is_adr_truncated
1844,"Балтийская ул., 14",NaN,"5-к. квартира, 167,3 м², 7/8 эт.",17.75,5,7,8,167.30,False,False,...,False,False,False,False,"Пятикомнатная квартира 167,3 м² на 7/8 этаже дома с большой кухней 30,2 м². Расположена в г. Севастополь, Ленинский район, ул. Балтийская, д.14. \n\nЭта квартира идеально подойдет для постоянного проживания большой и дружной семьи. \n\nПозвоните нам, чтобы узн",17750000,43,2265256436,"(44.590959, 33.5032396)",False
1962,"пос. городского типа Кача, ул. Авиаторов, 1Гк3",NaN,"1-к. апартаменты, 52 м², 4/6 эт.",7.80,1,4,6,52.00,False,True,...,False,False,False,False,"К продаже предлагаю чудесные апартаменты в Каче на самом берегу моря. Комплекс апартаментов «Наш Парус» — это уникальный по своему месторасположению объект курортного отдыха. Развита инфраструктура, отличная транспортная развязка. \nО самих апартаментах. Оч",7800000,68,2439435190,None,None
1645,"проспект Октябрьской Революции, 22/3",NaN,"3-к. квартира, 68 м², 2/9 эт.",9.80,3,2,9,68.00,False,False,...,False,False,False,False,"Продается уютная трехкомнатная квартира улучшенной планировки в одном из самых востребованных районов города. \n\nОбщая площадь квартиры 68 м. Кв. И жилой 39 кв. М. Три изолированные комнаты: 18 м. Кв, 12 м. Кв, 9 м. Кв., выходящие на обе стороны дома с прос",9800000,70,2473522762,"(44.5922607, 33.4568337)",True


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3953 entries, 0 to 3952
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   adr               3953 non-null   object 
 1   obj_name          872 non-null    object 
 2   title             3952 non-null   object 
 3   priceM            3953 non-null   float64
 4   nrooms            3953 non-null   int64  
 5   floor             3953 non-null   int64  
 6   nfloors           3953 non-null   int64  
 7   area              3953 non-null   float64
 8   is_studio         3953 non-null   bool   
 9   is_apartment      3953 non-null   bool   
 10  is_part           3953 non-null   bool   
 11  is_auction        3953 non-null   bool   
 12  is_openspace      3953 non-null   bool   
 13  is_SNT            3953 non-null   bool   
 14  is_last_floor     3953 non-null   bool   
 15  is_roof           3953 non-null   bool   
 16  description       3949 non-null   object 


In [8]:
df_ = df.query(
    '~(is_studio|is_apartment|is_part|is_auction|is_openspace|is_SNT|is_roof)'
    '&(nrooms==2)'
)

df_[['priceM']].describe().T    

,count,mean,std,min,25%,50%,75%,max
priceM,1038.00,8.94,3.31,2.65,6.70,8.19,10.50,36.00


In [9]:
cols = ['adr', 'priceM', 'nrooms', 'floor', 'nfloors', 'area', 'description', 'avito_id',]
df_[cols].sort_values(by=['priceM'])[:20]

,adr,priceM,nrooms,floor,nfloors,area,description,avito_id
3306,"с. Песчаное, ул. Верхний Городок, 2А",2.65,2,1,1,51.00,"2комнаты, ванна, туалет, кухня коридор, отопление газовое, вода, свет центральные, с участком земли, вход отдельный, с документами все в порядке, срочно, торг.",2538746691
3635,"с. Вилино, ул. Кирова, 2",2.90,2,2,2,38.30,"Двухкомнатная квартира 38,3 м2, на 2/2 этаже дома. Расположена в Республике Крым, Бахчисарайский район, с. Вилино, ул. Кирова, д. 2. \n\nПозвоните нам, чтобы узнать все детали! \n\nТехнические характеристики: \n— Адрес: Республике Крым, Бахчисарайский район, с.",2457693926
750,"с. Вилино, ул. Чапаева, 1",3.30,2,1,2,46.50,"Продам 2-х комнатную квартиру в селе Вилино, Бахчисарайского района. Квартира тёплая имеется газ. Расположена на первом этаже двухэтажного многоквартирного дома. Требуется косметический ремонт.",2499615097
2494,"Чернореченская ул., 90А",3.40,2,1,1,38.00,"Собственник Обменипотека Эксклюзивное Предложение Квартиры в Инкермане 38кВ. М, в одноэтажном трех Квартирном доме с отдельным входом, двором, въездом, парковка, и однокомнатная с удобствами отдельно стоящая 30 кв. М на участке 4 сотки (Возможно сдавать 15",2390401919
751,"с. Вилино, ул. Чапаева, 1",3.40,2,1,2,45.90,"Двухкомнатная квартира 45,9 м2 на 1/2 этаже дома. Расположена в Республике Крым, с. Вилино, ул. Чапаева, д. 1. \n\nПозвоните нам, чтобы узнать все детали! \n\nТехнические характеристики: \n— Адрес: Республика Крым, с. Вилино, ул. Чапаева, д. 1. \n— Год постройки",2393768041
3370,"с. Песчаное, ул. Верхний Городок, 31А",3.60,2,1,2,52.40,"Продам 2 комнатную квартиру, сделан косметический ремонт, до моря 3 км.",2552106501
3369,"с. Песчаное, ул. Верхний Городок, 31А",3.60,2,1,2,52.40,"Продам 2 комнатную квартиру, до моря 3 км. Комнаты изолированные, с/у раздельный, большой балкон. В квартире 2 кладовые. Во дворе имеются детская и спортивная площадки.",2502101506
3646,"Республика Крым, Бахчисарайский район, село Вилино, улица Чапаева, 13",3.75,2,1,2,55.00,"Квартира у моря! \n\nПредлагаем двухкомнатную квартиру в с. Вилино Бахчисарайского района. \n\nОбщая площадь квартиры 55 кв. М., комнаты изолированные. Индивидуальное Газовое Отопление! \n\nНа окнах и лоджии установлены металлопластиковые стеклопакеты. Просторна",2550214980
3620,"Узловая ул., 116",3.90,2,1,2,43.00,"Двухкомнатная квартира 43,2 м2, на первом этаже двухэтажного дома. Расположена в г. Инкерман, Балаклавский район, улица Узловая. \n\nКвартира отлично подойдет как для молодой семейной пары, так и для небольшой семьи с детьми. \n\nПозвоните нам, чтобы узнать вс",2520256143
3152,"4-й Хомутов пер., 3",4.00,2,2,2,75.30,"Уже в продаже ваша стильная квартира с панорамным видом, террасой и чистовой отделкой. Много открытого пространства, панорамные окна и дом стиле High-tech, который идеально интегрирован в окружающий ландшафт. В нем сочетается обилие открытого пространства",2509674860


----

In [10]:
# !pip install folium

In [10]:
import folium

In [17]:
df__ = df_[ ~df_['coo'].isnull() ]
df__ = df__.query('priceM<6.').sample(50)[['title','priceM','adr','coo','avito_id']]

m = folium.Map(location=(44.5764241, 33.4685554),zoom_start=12)

for d in df__.values:
    title,price,adr,coo,avito_id = d
    tooltip = f'{price}M&nbsp;|&nbsp;'+title+'&nbsp;| '+ adr
    popup = f'<a target="_blank" href="https://www.avito.ru/{avito_id}">{tooltip}</a>'
    folium.Marker(coo,popup=popup,tooltip=tooltip).add_to(m)
    
m